## Notebook

Add julia functions by calling

In [ ]:
include("Julia_functions/get_room.jl")


In a julia code env

In [1]:
using Gridap, GridapMakie, CairoMakie, FileIO

In [ ]:
u(x) = x[1]^2 + x[2]^2
f(x) = u(x)-4
g(x) = -2
    

domain = (-1, 1, -1, 1)
partition = nx, ny = 20,20
model = CartesianDiscreteModel(domain, partition) |> simplexify

Ω = Triangulation(model)                                # make Triangulation
fig = plot(Ω)
wireframe!(Ω, color=:black)                             # Plot edges of each element
scatter!(Ω, marker=:circle, markersize=20, color=:blue) # Plot mesh nodes
display(fig)                                            # show plot
save("Exercise3_task2_Triangulation.svg", fig)          # save for later


order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)     # no Dirichlet (/ everything Neumann)

Trialspace = TrialFESpace(Testspace)                    # make trial space based on test space

degree = 2*order                                        # degree of the method used for approximating integrals over Ω                   
dΩ = Measure(Ω,degree)                                  # make the measure dΩ

Γ = BoundaryTriangulation(model)                        # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)                                  # measure on Γ

a(u,ϕ) = ∫( ∇(ϕ)⊙∇(u)+u*ϕ)*dΩ                          # bilinear form
l(ϕ) = ∫( ϕ*f )*dΩ - ∫( ϕ*g )*dΓ                        # linear functional

op = AffineFEOperator(a,l,Trialspace,Testspace)         # make FE operator - affine because the PDE is affine

uh = solve(op)                                          # solve problem

fig, _ , plt = plot(Ω, uh, colormap=:plasma)            # make plot of u_h
wireframe!(Ω, color=:black, linewidth=1)                # add triangulation to plot
Colorbar(fig[1,2], plt)                                 # add color bar
display(fig)                                            # show plot
save("Exercise3_task2_Solution.svg", fig)               # save picture


e = u - uh                                              # error

fig, _ , plt = plot(Ω, e, colormap=:plasma)             # plot of error
wireframe!(Ω, color=:black, linewidth=1)                # add triangulation
Colorbar(fig[1,2], plt)                                 # add color bar
display(fig)                                            # show plot
save("Exercise3_task2_Error.svg", fig)                  # save picture

el2 = sqrt(sum( ∫( e*e )*dΩ ))                         # calculate ||u-u_h||_L2
print("The L2 error ||u-uₕ|| is $el2")